Программа, которая определяет положительный или отрицательный отзыв.

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import os
import urllib.request 
import collections
from collections import Counter
from pymystem3 import Mystem
from nltk.tokenize import WordPunctTokenizer
from sklearn.metrics import accuracy_score
from nltk import pos_tag
from nltk.corpus import stopwords
import pymorphy2
url = 'https://www.kinopoisk.ru/film/1236063/reviews/?status=good&ord=date&rnd=1602064565'
req = urllib.request.Request('https://www.kinopoisk.ru/film/1236063/reviews/ord/date/status/good/perpage/50/') 
response = urllib.request.urlopen(req) 
html_content_positive = response.read().decode('utf-8')


In [2]:
req = urllib.request.Request('https://www.kinopoisk.ru/film/1236063/reviews/?status=bad&ord=date&rnd=1602073722&perpage=50') 
response = urllib.request.urlopen(req) 
html_content_negative = response.read().decode('utf-8')


In [3]:
soup_positive = BeautifulSoup(html_content_positive, 'html.parser')
soup_negative = BeautifulSoup(html_content_negative, 'html.parser')
reviews_positive = soup_positive.find_all(class_='_reachbanner_')
reviews_negative = soup_negative.find_all(class_='_reachbanner_')

In [4]:
def reviews_clean(reviews):
    reviews_clean = []
    for review in reviews:
        reviews_clean.append(review.find_all(text=True))
    return reviews_clean

In [5]:
reviews_clean_positive = reviews_clean(reviews_positive)
reviews_clean_negative = reviews_clean(reviews_negative)


In [6]:
def thirty_reviews(review):
    thirty_reviews = []
    k = 0
    while k != 40:
        thirty_reviews.append(review[k])
        k += 1
    
    return thirty_reviews

In [7]:
thirty_positive_reviews = thirty_reviews(reviews_clean_positive)
thirty_negative_reviews = thirty_reviews(reviews_clean_negative)

In [8]:
ten_reviews = []
k = 1
while k != 6:
    ten_reviews.append(reviews_clean_positive[-k])
    ten_reviews.append(reviews_clean_negative[-k])
    k += 1

In [9]:
import nltk
def words(reviews):
    sentences = []
    for review in reviews: 
        myString = ' '.join(review)
        sentence = nltk.sent_tokenize(myString)
        sentences.append(sentence)
    clean_words = []

    for sentence in sentences:
        for sen in sentence:
            words = nltk.word_tokenize(sen)
            clean_words.append(words)
    return clean_words

In [10]:
words_from_positive = words(thirty_positive_reviews)
words_from_negative = words(thirty_negative_reviews)
words_from_ten = words(ten_reviews)

In [11]:
def lists(words):
    words_new=[]
    for lst in words:
        for el in lst:
            words_new.append(el)
    return words_new

In [12]:
positive = lists(words_from_positive)
negative = lists(words_from_negative)
ten = lists(words_from_ten)

In [13]:
def clean_list(words):
    import re
    for k in words:
        if len(k) < 2:
            words.remove(k)
        k = k.replace("-", "")
        re.sub(r"\d+", "", k, flags=re.UNICODE)
    return words

In [14]:
clean_positive = clean_list(positive)
clean_negative = clean_list(negative)

In [15]:
def token_lower_tag(words):
    tokens = []
    for i in words:
        tokens.append(i.lower())
    return tokens

In [16]:
lower_positive = token_lower_tag(clean_positive)
lower_negative = token_lower_tag(clean_negative)

In [17]:
def lemmas(rev):
    morph = pymorphy2.MorphAnalyzer()
    new = []
    for word in rev:
        p = morph.parse(word)[0]
        new.append(p.normal_form)
    return new

In [18]:
lemmas_positive = lemmas(lower_positive)
lemmas_negative = lemmas(lower_negative)

In [23]:
russian_stopwords = stopwords.words("russian")
def delete_stopwords(review_1,stopwords):
    result=list(set(review_1) ^ set(stopwords))
    return result

In [24]:
new_positive = delete_stopwords(lemmas_positive,russian_stopwords)
new_negative = delete_stopwords(lemmas_negative,russian_stopwords)

In [27]:
from collections import Counter
couner_positive = Counter(new_positive)
couner_negative = Counter(new_negative)

In [28]:
most_common_positive = [k[0] for k in couner_positive.most_common(100)]
most_common_negative = [k[0] for k in couner_negative.most_common(100)]

In [29]:
for k in most_common_positive:
    for l in most_common_negative:
        if k == l:
            most_common_positive.remove(l)

In [30]:
for k in most_common_negative:
    for l in most_common_positive:
        if k == l:
            most_common_negative.remove(l)

In [31]:
def lemm_review(your_review):
    from pymystem3 import Mystem
    m = Mystem()
    new = ''.join(your_review)
    lemmas = m.lemmatize(new)
    for k in lemmas:
        if len(k) < 3:
            lemmas.remove(k)
    return lemmas

In [32]:
def che_review(your_review):
    x = 0
    c = 0
    for word in your_review:
        if word in most_common_positive:
            x +=1
        elif word in most_common_negative:
            c +=1
        else:
            pass
    if x > c:
        return 'positive'
    else:
        return 'negative'

In [33]:
che_review(lemm_review(ten_reviews[2]))

'negative'

Улучшения:
1) Увеличить выборку отзывов, чтобы получить больше данных для анализа
2) Использовать дополнительные фичи: например, word2vec или какие-то другие nlp-модели